### NER, Sentiment, Summary, Classification

In [1]:
import  numpy as np
import pandas  as pd

In [60]:
df = pd.read_csv('bbc_data.csv')
df

data         labels
0     Musicians to tackle US red tape  Musicians gro...  entertainment
1     U2s desire to be number one  U2, who have won ...  entertainment
2     Rocker Doherty in on-stage fight  Rock singer ...  entertainment
3     Snicket tops US box office chart  The film ada...  entertainment
4     Oceans Twelve raids box office  Oceans Twelve,...  entertainment
...                                                 ...            ...
2220  Warning over Windows Word files  Writing a Mic...           tech
2221  Fast lifts rise into record books  Two high-sp...           tech
2222  Nintendo adds media playing to DS  Nintendo is...           tech
2223  Fast moving phone viruses appear  Security fir...           tech
2224  Hacker threat to Apples iTunes  Users of Apple...           tech

[2225 rows x 2 columns]

In [61]:
df = df.sample(frac=0.05)

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111 entries, 1963 to 1161
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   data    111 non-null    object
 1   labels  111 non-null    object
dtypes: object(2)
memory usage: 2.6+ KB


In [63]:
df.labels.value_counts()

sport            27
tech             22
business         21
politics         21
entertainment    20
Name: labels, dtype: int64

In [64]:
import nltk
from nltk.corpus import stopwords
# from nltk.stem.snowball import  SnowballStemmer
from nltk.stem import WordNetLemmatizer

In [65]:
def clean_text(x):
    temp=[]
    lem =[]
    words = nltk.word_tokenize(x)
    stop_words = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    for word in words:
        word = word.lower()
        if word in stop_words or not(word.isalpha()):  #only text and no stop words
            pass
        else:
            temp.append(word)
    for i in temp:
        lem.append(lemmatizer.lemmatize(i))
    return ' '.join(lem)

In [66]:
df['Cleaned_Data'] = df.data.apply(clean_text)

In [67]:
df

data         labels  \
1963  Spam e-mails tempt net shoppers  Computer user...           tech   
634   Cairn Energy in Indian gas find  Shares in Cai...       business   
538   Yangtze Electrics profits double  Yangtze Elec...       business   
2211  Millions buy MP3 players in US  One in 10 adul...           tech   
86    Help for indies in download sales  A campaign ...  entertainment   
...                                                 ...            ...   
275   No UK premiere for Rings musical  The producer...  entertainment   
893   Euro firms miss out on optimism  More than 90%...       business   
150   No charges against TVs Cosby  US comedian Bill...  entertainment   
698   Fosters buys stake in winemaker  Australian br...       business   
1161  Off-colour Gardener storms to win  Britains Ja...          sport   

                                           Cleaned_Data  
1963  spam tempt net shopper computer user across wo...  
634   cairn energy indian gas find share cairn energ...  
538   yangtze electric profit double yangtze electri...  
2211  million buy player u one adult american equiva...  
86    help indie download sale campaign launched hel...  
...                                                 ...  
275   uk premiere ring musical producer behind lord ...  
893   euro firm miss optimism large company around w...  
150   charge tv cosby u comedian bill cosby face cha...  
698   foster buy stake winemaker australian brewer f...  
1161  gardener storm win britain jason gardener shoo...  

[111 rows x 3 columns]

### NER

In [6]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [7]:
!pip3 install torch torchvision torchaudio

Defaulting to user installation because normal site-packages is not writeable


In [68]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)



Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity': 'B-PER', 'score': 0.9990139, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


In [69]:
df['NER'] = df.data.apply(nlp)

In [70]:
df

data         labels  \
1963  Spam e-mails tempt net shoppers  Computer user...           tech   
634   Cairn Energy in Indian gas find  Shares in Cai...       business   
538   Yangtze Electrics profits double  Yangtze Elec...       business   
2211  Millions buy MP3 players in US  One in 10 adul...           tech   
86    Help for indies in download sales  A campaign ...  entertainment   
...                                                 ...            ...   
275   No UK premiere for Rings musical  The producer...  entertainment   
893   Euro firms miss out on optimism  More than 90%...       business   
150   No charges against TVs Cosby  US comedian Bill...  entertainment   
698   Fosters buys stake in winemaker  Australian br...       business   
1161  Off-colour Gardener storms to win  Britains Ja...          sport   

                                           Cleaned_Data  \
1963  spam tempt net shopper computer user across wo...   
634   cairn energy indian gas find share cairn energ...   
538   yangtze electric profit double yangtze electri...   
2211  million buy player u one adult american equiva...   
86    help indie download sale campaign launched hel...   
...                                                 ...   
275   uk premiere ring musical producer behind lord ...   
893   euro firm miss optimism large company around w...   
150   charge tv cosby u comedian bill cosby face cha...   
698   foster buy stake winemaker australian brewer f...   
1161  gardener storm win britain jason gardener shoo...   

                                                    NER  
1963  [{'entity': 'B-ORG', 'score': 0.998735, 'index...  
634   [{'entity': 'B-ORG', 'score': 0.9987987, 'inde...  
538   [{'entity': 'B-ORG', 'score': 0.9992798, 'inde...  
2211  [{'entity': 'B-MISC', 'score': 0.9487226, 'ind...  
86    [{'entity': 'B-ORG', 'score': 0.9963684, 'inde...  
...                                                 ...  
275   [{'entity': 'B-LOC', 'score': 0.9977362, 'inde...  
893   [{'entity': 'B-LOC', 'score': 0.978403, 'index...  
150   [{'entity': 'B-PER', 'score': 0.8420079, 'inde...  
698   [{'entity': 'B-ORG', 'score': 0.99589145, 'ind...  
1161  [{'entity': 'B-PER', 'score': 0.97326875, 'ind...  

[111 rows x 4 columns]

### Sentiment Analysis

In [71]:
from transformers import pipeline
model_path =  f"cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)
sentiment_task("Covid cases are increasing fast!")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'label': 'negative', 'score': 0.723576545715332}]

In [72]:
df.data.iloc[0]

'Spam e-mails tempt net shoppers  Computer users across the world continue to ignore security warnings about spam e-mails and are being lured into buying goods, a report suggests.  More than a quarter have bought software through spam e-mails and 24% have bought clothes or jewellery. As well as profiting from selling goods or services and driving advertising traffic, organised crime rings can use spam to glean personal information. The Business Software Alliance (BSA) warned that people should "stay alert". "Many online consumers dont consider the true motives of spammers," said Mike Newton, a spokesperson for the BSA which commissioned the survey.  "By selling software that appears to be legitimate in genuine looking packaging or through sophisticated websites, spammers are hiding spyware without consumers knowledge. "Once the software is installed on PCs and networks, information that is given over the internet can be obtained and abused."  The results also showed that the proportion

In [73]:
sentiment_task(df.data.iloc[0])

[{'label': 'negative', 'score': 0.6629323363304138}]

In [74]:
# sentiment_task("Covid cases are increasing fast!")[0]['label']

In [75]:
for index,i in enumerate(df.data):
    try:
        print(index,':',len(i),': ',sentiment_task(i))
    except:
        print(index,':error:',len(i))
                         

0 : 2283 :  [{'label': 'negative', 'score': 0.6629323363304138}]
1 : 1009 :  [{'label': 'neutral', 'score': 0.7790276408195496}]
2 : 1636 :  [{'label': 'positive', 'score': 0.8211090564727783}]
3 : 1842 :  [{'label': 'neutral', 'score': 0.5662821531295776}]
4 : 1142 :  [{'label': 'neutral', 'score': 0.5653825998306274}]
5 : 890 :  [{'label': 'neutral', 'score': 0.8240184187889099}]
6 : 1052 :  [{'label': 'neutral', 'score': 0.5025995969772339}]
7 : 2417 :  [{'label': 'positive', 'score': 0.6612546443939209}]
8 :error: 2635
9 :error: 4687
10 : 1517 :  [{'label': 'neutral', 'score': 0.8198977112770081}]
11 :error: 5105
12 : 2122 :  [{'label': 'positive', 'score': 0.6230218410491943}]
13 :error: 2381
14 : 1702 :  [{'label': 'neutral', 'score': 0.7200088500976562}]
15 : 769 :  [{'label': 'neutral', 'score': 0.8723119497299194}]
16 :error: 4168
17 :error: 2994
18 :error: 2349
19 :error: 2414
20 :error: 2911
21 : 1291 :  [{'label': 'neutral', 'score': 0.7166378498077393}]
22 :error: 2894
23 

In [76]:
for index,i in enumerate(df.Cleaned_Data):
    try:
        print(index,':',len(i),': ',sentiment_task(i))
    except:
        print(index,':error:',len(i))
                         

0 : 1528 :  [{'label': 'negative', 'score': 0.590985119342804}]
1 : 714 :  [{'label': 'neutral', 'score': 0.7133961319923401}]
2 : 1051 :  [{'label': 'neutral', 'score': 0.7179546356201172}]
3 : 1108 :  [{'label': 'neutral', 'score': 0.7903138995170593}]
4 : 828 :  [{'label': 'neutral', 'score': 0.6129865646362305}]
5 : 612 :  [{'label': 'neutral', 'score': 0.7410463690757751}]
6 : 724 :  [{'label': 'neutral', 'score': 0.7823981642723083}]
7 : 1509 :  [{'label': 'neutral', 'score': 0.5686325430870056}]
8 : 1796 :  [{'label': 'negative', 'score': 0.6281067132949829}]
9 : 2923 :  [{'label': 'neutral', 'score': 0.7443824410438538}]
10 : 1019 :  [{'label': 'neutral', 'score': 0.8861977458000183}]
11 :error: 3179
12 : 1313 :  [{'label': 'positive', 'score': 0.5274903178215027}]
13 : 1646 :  [{'label': 'neutral', 'score': 0.8191014528274536}]
14 : 1045 :  [{'label': 'negative', 'score': 0.49536797404289246}]
15 : 466 :  [{'label': 'neutral', 'score': 0.8031299710273743}]
16 : 2816 :  [{'labe

In [77]:
def sentiment_analysis(text):
    sentiment = sentiment_task(text[:2000]) # limiting text to 2000(the is random, based on intution?) characters due to limit
    return sentiment[0]['label']

In [78]:
df['sentiment_cleaned_data'] = df.Cleaned_Data.apply(sentiment_analysis) #Sentiment on cleaned data

In [79]:
df['sentiment_original_data'] = df.data.apply(sentiment_analysis) #Sentiment on original data

In [80]:
df

data         labels  \
1963  Spam e-mails tempt net shoppers  Computer user...           tech   
634   Cairn Energy in Indian gas find  Shares in Cai...       business   
538   Yangtze Electrics profits double  Yangtze Elec...       business   
2211  Millions buy MP3 players in US  One in 10 adul...           tech   
86    Help for indies in download sales  A campaign ...  entertainment   
...                                                 ...            ...   
275   No UK premiere for Rings musical  The producer...  entertainment   
893   Euro firms miss out on optimism  More than 90%...       business   
150   No charges against TVs Cosby  US comedian Bill...  entertainment   
698   Fosters buys stake in winemaker  Australian br...       business   
1161  Off-colour Gardener storms to win  Britains Ja...          sport   

                                           Cleaned_Data  \
1963  spam tempt net shopper computer user across wo...   
634   cairn energy indian gas find share cairn energ...   
538   yangtze electric profit double yangtze electri...   
2211  million buy player u one adult american equiva...   
86    help indie download sale campaign launched hel...   
...                                                 ...   
275   uk premiere ring musical producer behind lord ...   
893   euro firm miss optimism large company around w...   
150   charge tv cosby u comedian bill cosby face cha...   
698   foster buy stake winemaker australian brewer f...   
1161  gardener storm win britain jason gardener shoo...   

                                                    NER  \
1963  [{'entity': 'B-ORG', 'score': 0.998735, 'index...   
634   [{'entity': 'B-ORG', 'score': 0.9987987, 'inde...   
538   [{'entity': 'B-ORG', 'score': 0.9992798, 'inde...   
2211  [{'entity': 'B-MISC', 'score': 0.9487226, 'ind...   
86    [{'entity': 'B-ORG', 'score': 0.9963684, 'inde...   
...                                                 ...   
275   [{'entity': 'B-LOC', 'score': 0.9977362, 'inde...   
893   [{'entity': 'B-LOC', 'score': 0.978403, 'index...   
150   [{'entity': 'B-PER', 'score': 0.8420079, 'inde...   
698   [{'entity': 'B-ORG', 'score': 0.99589145, 'ind...   
1161  [{'entity': 'B-PER', 'score': 0.97326875, 'ind...   

     sentiment_cleaned_data sentiment_original_data  
1963               negative                negative  
634                 neutral                 neutral  
538                 neutral                positive  
2211                neutral                 neutral  
86                  neutral                 neutral  
...                     ...                     ...  
275                 neutral                 neutral  
893                negative                 neutral  
150                 neutral                 neutral  
698                 neutral                 neutral  
1161               positive                positive  

[111 rows x 6 columns]

### Text Summarization

In [81]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("suriya7/bart-finetuned-text-summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("suriya7/bart-finetuned-text-summarization")

def generate_summary(text):
    inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_new_tokens=100, do_sample=False)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

text_to_summarize = """Now, there is no doubt that one of the most important aspects of any Pixel phone is its camera.
And there might be good news for all camera lovers. Rumours have suggested that the Pixel 9 could come with a telephoto lens,
improving its photography capabilities even further. Google will likely continue to focus on using AI to enhance its camera performance,
in order to make sure that Pixel phones remain top contenders in the world of mobile photography."""
summary = generate_summary(text_to_summarize)
print(summary)


Google is rumoured to be about to unveil its next-generation Pixel smartphone, the Google Pixel 9, which is expected to come with a bigger screen and a new AI-powered camera, as well as a new range of sensors to improve the quality of the camera.


In [82]:
df['summary'] = df.data.apply(generate_summary)

In [83]:
df

data         labels  \
1963  Spam e-mails tempt net shoppers  Computer user...           tech   
634   Cairn Energy in Indian gas find  Shares in Cai...       business   
538   Yangtze Electrics profits double  Yangtze Elec...       business   
2211  Millions buy MP3 players in US  One in 10 adul...           tech   
86    Help for indies in download sales  A campaign ...  entertainment   
...                                                 ...            ...   
275   No UK premiere for Rings musical  The producer...  entertainment   
893   Euro firms miss out on optimism  More than 90%...       business   
150   No charges against TVs Cosby  US comedian Bill...  entertainment   
698   Fosters buys stake in winemaker  Australian br...       business   
1161  Off-colour Gardener storms to win  Britains Ja...          sport   

                                           Cleaned_Data  \
1963  spam tempt net shopper computer user across wo...   
634   cairn energy indian gas find share cairn energ...   
538   yangtze electric profit double yangtze electri...   
2211  million buy player u one adult american equiva...   
86    help indie download sale campaign launched hel...   
...                                                 ...   
275   uk premiere ring musical producer behind lord ...   
893   euro firm miss optimism large company around w...   
150   charge tv cosby u comedian bill cosby face cha...   
698   foster buy stake winemaker australian brewer f...   
1161  gardener storm win britain jason gardener shoo...   

                                                    NER  \
1963  [{'entity': 'B-ORG', 'score': 0.998735, 'index...   
634   [{'entity': 'B-ORG', 'score': 0.9987987, 'inde...   
538   [{'entity': 'B-ORG', 'score': 0.9992798, 'inde...   
2211  [{'entity': 'B-MISC', 'score': 0.9487226, 'ind...   
86    [{'entity': 'B-ORG', 'score': 0.9963684, 'inde...   
...                                                 ...   
275   [{'entity': 'B-LOC', 'score': 0.9977362, 'inde...   
893   [{'entity': 'B-LOC', 'score': 0.978403, 'index...   
150   [{'entity': 'B-PER', 'score': 0.8420079, 'inde...   
698   [{'entity': 'B-ORG', 'score': 0.99589145, 'ind...   
1161  [{'entity': 'B-PER', 'score': 0.97326875, 'ind...   

     sentiment_cleaned_data sentiment_original_data  \
1963               negative                negative   
634                 neutral                 neutral   
538                 neutral                positive   
2211                neutral                 neutral   
86                  neutral                 neutral   
...                     ...                     ...   
275                 neutral                 neutral   
893                negative                 neutral   
150                 neutral                 neutral   
698                 neutral                 neutral   
1161               positive                positive   

                                                summary  
1963  More than a third of people in six countries a...  
634   Shares in Cairn Energy have risen after the co...  
538   China's Yangtze River hydroelectric power comp...  
2211  One in 10 adults in the US have an MP3 player,...  
86    The UK's independent music industry has launch...  
...                                                 ...  
275   No UK premiere for Lord of the Rings musical h...  
893   Companies in Western Europe are more optimisti...  
150   US comedian Bill Cosby will not face charges i...  
698   Australia's Fosters has said it is in talks wi...  
1161  Great Britain's Jason Gardener has won his fir...  

[111 rows x 7 columns]

### Sample Output

In [85]:
df.data.iloc[0]

'Spam e-mails tempt net shoppers  Computer users across the world continue to ignore security warnings about spam e-mails and are being lured into buying goods, a report suggests.  More than a quarter have bought software through spam e-mails and 24% have bought clothes or jewellery. As well as profiting from selling goods or services and driving advertising traffic, organised crime rings can use spam to glean personal information. The Business Software Alliance (BSA) warned that people should "stay alert". "Many online consumers dont consider the true motives of spammers," said Mike Newton, a spokesperson for the BSA which commissioned the survey.  "By selling software that appears to be legitimate in genuine looking packaging or through sophisticated websites, spammers are hiding spyware without consumers knowledge. "Once the software is installed on PCs and networks, information that is given over the internet can be obtained and abused."  The results also showed that the proportion

In [86]:
df.Cleaned_Data.iloc[0]

'spam tempt net shopper computer user across world continue ignore security warning spam lured buying good report suggests quarter bought software spam bought clothes jewellery well profiting selling good service driving advertising traffic organised crime ring use spam glean personal information business software alliance bsa warned people stay alert many online consumer dont consider true motif spammer said mike newton spokesperson bsa commissioned survey selling software appears legitimate genuine looking packaging sophisticated website spammer hiding spyware without consumer knowledge software installed pc network information given internet obtained abused result also showed proportion people reading admitting reading taking advantage adult entertainment spam low one research covered people six country attitude towards junk revealed brazilian likely read spam third read unsolicited junk buy good service receiving spam french second likely buy something briton taking advantage produ

In [96]:
print('SUMMARY:\n',df.summary.iloc[0])
print('\nSENTIMENT:\n', df.sentiment_original_data.iloc[0])

SUMMARY:
 More than a third of people in six countries are buying goods or services after receiving unsolicited junk e-mails, according to a survey of more than 6,000 people. Spammers are "hiding spyware without consumers knowledge", warns the Business Software Alliance (BSA).

SENTIMENT:
 negative


In [88]:
#NER
for i in df.NER.iloc[0]:
    print(i['word'],' : ', i['entity'] )

Business  :  B-ORG
Software  :  I-ORG
Alliance  :  I-ORG
BS  :  B-ORG
##A  :  I-ORG
Mike  :  B-PER
Newton  :  I-PER
BS  :  B-ORG
##A  :  I-ORG
Brazilian  :  B-MISC
French  :  B-MISC
B  :  B-MISC
##rito  :  B-MISC
